# This project is a code work that allows chatting with the text in the pdf file. It is integrated with artificial intelligence.

<img src="Wavy_Edu-01_Single-06.jpg" alt="pdf chat" style="width:200px;height:200px;">

In [7]:
#Required packages
#!pip install openai
#!pip install PyPDF2
#!pip install tiktoken
#!pip install faiss-cpu
#!pip install langchain

In [8]:
from PyPDF2 import PdfReader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain import FAISS
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAI
from langchain.callbacks import get_openai_callback
import os
import openai
os.environ["OPENAI_API_KEY"] ="xxxx"

In [9]:
def process_text(text):
    # Split the text into chunks using langchain
    text_splitter = CharacterTextSplitter(
        separator="\n",
        chunk_size=1000,
        chunk_overlap=200,
        length_function=len
    )
    chunks = text_splitter.split_text(text)

    # Convert the chunks of text into embeddings to form a knowledge base
    embeddings = OpenAIEmbeddings()
    knowledgeBase = FAISS.from_texts(chunks, embeddings)

    return knowledgeBase

In [10]:
pdf_reader = PdfReader('Bilgi ve Iletisim Güvenligi Rehberi.pdf')

In [11]:
text = ""
for page in pdf_reader.pages:
    text += page.extract_text()

# Create the knowledge base object
knowledgeBase = process_text(text)

In [12]:
def soru(query):
    docs = knowledgeBase.similarity_search(query)
    llm = OpenAI()
    chain = load_qa_chain(llm, chain_type='stuff')
    with get_openai_callback() as cost:
        response = chain.run(input_documents=docs, question=query)
    print(response)

In [13]:
soru('Does this study contain extraction experiments?')

 No, this study does not contain extraction experiments. It contains descriptions of security measures and audit methods to be used to prevent vulnerabilities related to command injection attacks in database access.


In [14]:
soru('Bu makale ne hakkında?')

 Bu makale, 2019/12 Sayılı Cumhurbaşkanlığı Genelgesi'nde yer alan 21 adet tedbirin Bilgi ve İletişim Güvenliği Rehberi başlıklarıyla eşleşmesini açıklar.


In [15]:
soru('Siber güvenlik tedbirlerinden hangisi yanlıştır. l7 koruma , ddos , güvenlik duvarı , home security?')

 Hiçbiri yanlış değil.
